In [6]:
import io
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from pathlib import Path
from PIL import Image 

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
path = Path("analyze.ipynb")
project_path = str(path.parent.absolute().parent)

In [8]:
def read_csv(path: str) -> pd.DataFrame:
    return pd.read_csv(os.path.join(project_path, path))

def read_image(image_name: str) -> Image:
    with h5py.File(os.path.join(project_path, r"data\raw", "train-image.hdf5"), "r") as f:
        im = Image.open(io.BytesIO(f[image_name][()]))
    return im

def load_hdf5(path: str):
    with h5py.File(os.path.join(project_path, path), "r") as f:
        file = f
    return file

In [9]:
train_hdf5 = load_hdf5("data/raw/train-image.hdf5")
train_hdf5

<Closed HDF5 file>

In [13]:
with h5py.File(os.path.join(project_path, r"data/raw/train-image.hdf5"), "r") as f:
    keys = list(f.keys())[:10]
    labels = [1] * 10
    data = []
    for i in keys:
        im = Image.open(io.BytesIO(f[i][()]))
        im = im.resize((128, 128))
        pix = np.array(im)
        data.append(pix)
        print(pix.shape)
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))

dataset


(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)


<_TensorSliceDataset element_spec=(TensorSpec(shape=(128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [15]:
def generator():
    with h5py.File(os.path.join(project_path, r"data/raw/train-image.hdf5"), "r") as f:
        keys = list(f.keys())[:10]
        labels = [1] * 10
        data = []
        for i in keys:
            im = Image.open(io.BytesIO(f[i][()]))
            im = im.resize((128, 128))
            pix = np.array(im)
            data.append(pix)
            yield pix

In [16]:
next(iter(generator()))

array([[[180, 146, 134],
        [180, 146, 134],
        [180, 146, 134],
        ...,
        [192, 154, 145],
        [190, 152, 143],
        [189, 152, 143]],

       [[185, 151, 139],
        [184, 150, 138],
        [184, 149, 137],
        ...,
        [189, 150, 142],
        [188, 150, 141],
        [186, 149, 140]],

       [[192, 156, 144],
        [191, 155, 143],
        [190, 154, 142],
        ...,
        [183, 145, 136],
        [182, 144, 135],
        [183, 145, 136]],

       ...,

       [[191, 150, 144],
        [184, 143, 137],
        [178, 137, 131],
        ...,
        [176, 133, 120],
        [175, 131, 119],
        [175, 132, 119]],

       [[190, 149, 143],
        [184, 143, 137],
        [178, 138, 130],
        ...,
        [174, 130, 117],
        [175, 131, 118],
        [176, 132, 119]],

       [[188, 147, 141],
        [183, 142, 136],
        [177, 137, 129],
        ...,
        [174, 130, 117],
        [174, 130, 117],
        [175, 131, 118]]

In [21]:
ds = tf.data.Dataset.from_generator(generator,
                               output_signature=(
                                   tf.TensorSpec(shape=(64, 64, 3), dtype=tf.uint8),
                                   tf.TensorSpec(shape=(), dtype=tf.int32)
                               ))
ds

<_FlatMapDataset element_spec=(TensorSpec(shape=(64, 64, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [22]:
next(iter(ds))

2024-07-10 18:19:40.068082: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.uint8, tf.int32), but the yielded element was [[[180 146 134]
  [180 146 134]
  [180 146 134]
  ...
  [192 154 145]
  [190 152 143]
  [189 152 143]]

 [[185 151 139]
  [184 150 138]
  [184 149 137]
  ...
  [189 150 142]
  [188 150 141]
  [186 149 140]]

 [[192 156 144]
  [191 155 143]
  [190 154 142]
  ...
  [183 145 136]
  [182 144 135]
  [183 145 136]]

 ...

 [[191 150 144]
  [184 143 137]
  [178 137 131]
  ...
  [176 133 120]
  [175 131 119]
  [175 132 119]]

 [[190 149 143]
  [184 143 137]
  [178 138 130]
  ...
  [174 130 117]
  [175 131 118]
  [176 132 119]]

 [[188 147 141]
  [183 142 136]
  [177 137 129]
  ...
  [174 130 117]
  [174 130 117]
  [175 131 118]]].
Traceback (most recent call last):

  File "/home/peter/miniconda3/envs/isic/lib/python3.10/site-packages/tensorfl

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.uint8, tf.int32), but the yielded element was [[[180 146 134]
  [180 146 134]
  [180 146 134]
  ...
  [192 154 145]
  [190 152 143]
  [189 152 143]]

 [[185 151 139]
  [184 150 138]
  [184 149 137]
  ...
  [189 150 142]
  [188 150 141]
  [186 149 140]]

 [[192 156 144]
  [191 155 143]
  [190 154 142]
  ...
  [183 145 136]
  [182 144 135]
  [183 145 136]]

 ...

 [[191 150 144]
  [184 143 137]
  [178 137 131]
  ...
  [176 133 120]
  [175 131 119]
  [175 132 119]]

 [[190 149 143]
  [184 143 137]
  [178 138 130]
  ...
  [174 130 117]
  [175 131 118]
  [176 132 119]]

 [[188 147 141]
  [183 142 136]
  [177 137 129]
  ...
  [174 130 117]
  [174 130 117]
  [175 131 118]]].
Traceback (most recent call last):

  File "/home/peter/miniconda3/envs/i